# ANN Model Training for Credit Card Fraud Detection

This notebook walks through the complete process of training an Artificial Neural Network (ANN) to detect fraudulent credit card transactions. We will use the popular [Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud) dataset from Kaggle.

**The process is divided into five main steps:**
1.  **Setup and Data Download**: We'll set up the Kaggle API and download the dataset.
2.  **Data Preprocessing**: We'll load, explore, and prepare the data for our model.
3.  **ANN Model Building**: We'll define the architecture of our neural network.
4.  **Model Training and Evaluation**: We'll train the model on our data, handling the class imbalance, and then evaluate its performance.
5.  **Saving the Model**: We'll save the final trained model for use in our backend application.

### Step 1: Setup and Data Download

First, we need to install the necessary libraries and set up the Kaggle API to download our dataset directly. 

**Important:** Before running the cell below, you must upload your `kaggle.json` API token file. You can do this by clicking the 'folder' icon on the left sidebar in Google Colab and uploading the file.

In [2]:
!pip install -q kaggle pandas numpy tensorflow scikit-learn matplotlib seaborn

import os
import zipfile

# Set up Kaggle directory and permissions
if not os.path.exists('~/.kaggle'):
    os.makedirs(os.path.expanduser('~/.kaggle'))

# Make sure kaggle.json is in the right place
# If running locally, ensure ~/.kaggle/kaggle.json exists.
# If in Colab, upload kaggle.json to the session storage.
if os.path.exists('kaggle.json'):
    os.system('cp kaggle.json ~/.kaggle/')
    os.system('chmod 600 ~/.kaggle/kaggle.json')

# Create data directory
if not os.path.exists('data'):
    os.makedirs('data')

# Download and unzip the dataset
print("Downloading dataset from Kaggle...")
!kaggle datasets download -d mlg-ulb/creditcardfraud -p data/ --unzip
print("Dataset downloaded and extracted successfully.")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Shashvat Singh/.kaggle'

### Step 2: Load and Preprocess the Data

Now that we have the data, we'll load it into a pandas DataFrame. The most critical preprocessing step here is to scale the `Time` and `Amount` columns, as their values are not on the same scale as the other anonymized PCA features (`V1` to `V28`).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_and_prepare_data(filepath='data/creditcard.csv'):
    """
    Loads the dataset and prepares it for training.
    """
    try:
        df = pd.read_csv(filepath)
        print("Dataset loaded successfully.")
        print(f"Dataset shape: {df.shape}")
        print("\nFirst 5 rows of the data:")
        print(df.head())
        
        # Scale 'Amount' and 'Time' features
        scaler = StandardScaler()
        df['scaled_amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
        df['scaled_time'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))
        df = df.drop(['Time', 'Amount'], axis=1)
        
        # Reorder columns to have 'Class' at the end
        df.insert(0, 'scaled_amount_col', df.pop('scaled_amount'))
        df.insert(1, 'scaled_time_col', df.pop('scaled_time'))
        
        return df
    except FileNotFoundError:
        print(f"Error: The file {filepath} was not found.")
        return None

transaction_df = load_and_prepare_data()

# Define features (X) and target (y)
X = transaction_df.drop('Class', axis=1)
y = transaction_df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nData preprocessing and splitting complete.")
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

### Step 3: Build the ANN Model

Here, we define the architecture for our neural network. It's a simple sequential model with a few dense layers and dropout layers to prevent overfitting. The final layer uses a `sigmoid` activation function, which is perfect for binary classification as it outputs a probability between 0 and 1.

In [ ]:
import tensorflow as tf

def build_fraud_detection_model(input_shape):
    """
    Builds and compiles the ANN model architecture.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

# The input shape is the number of feature columns
input_shape = X_train.shape[1]
fraud_classifier_model = build_fraud_detection_model(input_shape)

print("ANN model built successfully.")
fraud_classifier_model.summary()

### Step 4: Train and Evaluate the Model

This is the most crucial step. We will train the model, but we need to handle the severe class imbalance in the dataset. We'll use `class_weight` to tell the model to pay significantly more attention to the fraudulent transactions (the minority class).

After training, we will evaluate the model's performance using a classification report and a confusion matrix to see how well it distinguishes between fraudulent and legitimate transactions.

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Handle class imbalance by calculating class weights
neg, pos = np.bincount(y_train)
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Class weights: Fraudulent (1): {weight_for_1:.2f}, Non-Fraudulent (0): {weight_for_0:.2f}\n")

# Set up early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

print("Starting model training...")
history = fraud_classifier_model.fit(
    X_train,
    y_train,
    batch_size=2048,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stopping],
    class_weight=class_weight
)

print("\n--- Model Evaluation ---")
predictions_prob = fraud_classifier_model.predict(X_test)
predictions = (predictions_prob > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names=['Legitimate', 'Fraud']))

print("ROC AUC Score:", roc_auc_score(y_test, predictions_prob))

# Display Confusion Matrix
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Legitimate', 'Fraud'], yticklabels=['Legitimate', 'Fraud'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

### Step 5: Save the Trained Model

Finally, we'll save our trained model to a file named `fraud_detection_ann.h5`. This file can then be loaded by our Flask backend to make live predictions.

In [ ]:
def save_trained_model(model, path='models/'):
    """
    Saves the final trained model to the specified path.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    
    model_path = os.path.join(path, 'fraud_detection_ann.h5')
    model.save(model_path)
    print(f"\nModel saved successfully to {model_path}")

save_trained_model(fraud_classifier_model)